In [2]:
import os
import numpy as np
import pandas as pd
import datetime
from datetime import date

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.preprocessing import MinMaxScaler

DATA_ROOT = "data/"

In [67]:
dfoff = pd.read_csv(os.path.join(DATA_ROOT,'train_offline.csv'))
#dfoff = dfoff.iloc[:5000]
dftest = pd.read_csv(os.path.join(DATA_ROOT,'test_offline.csv'))
#dftest = dftest.iloc[:5000]
dftest = dftest[~dftest.Coupon_id.isna()]
dftest.reset_index(drop=True, inplace=True)
print(dfoff.shape)
print(dftest.shape)
dfoff.head(5)

(1160742, 7)
(306313, 6)


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN


In [68]:
DistanceFilling_UM = dfoff.groupby(['User_id','Merchant_id'])['Distance'].mean().reset_index()
DistanceFilling_UM.columns = ['User_id','Merchant_id','DistanceFilling_UM']
DistanceFilling_U = dfoff.groupby(['User_id'])['Distance'].mean().reset_index()
DistanceFilling_U.columns = ['User_id','DistanceFilling_U']
DistanceFilling_M = dfoff.groupby(['Merchant_id'])['Distance'].mean().reset_index()
DistanceFilling_M.columns = ['Merchant_id','DistanceFilling_M']

DistanceFilling_UM = dftest.groupby(['User_id','Merchant_id'])['Distance'].mean().reset_index()
DistanceFilling_UM.columns = ['User_id','Merchant_id','DistanceFilling_UM']
DistanceFilling_U = dftest.groupby(['User_id'])['Distance'].mean().reset_index()
DistanceFilling_U.columns = ['User_id','DistanceFilling_U']
DistanceFilling_M = dftest.groupby(['Merchant_id'])['Distance'].mean().reset_index()
DistanceFilling_M.columns = ['Merchant_id','DistanceFilling_M']

dfoff = pd.merge(dfoff,DistanceFilling_UM,on = ['User_id','Merchant_id'], how = 'left')
dfoff = pd.merge(dfoff,DistanceFilling_U,on = ['User_id'], how = 'left')
dfoff = pd.merge(dfoff,DistanceFilling_M,on = ['Merchant_id'], how = 'left')


dftest = pd.merge(dftest,DistanceFilling_UM,on = ['User_id','Merchant_id'], how = 'left')
dftest = pd.merge(dftest,DistanceFilling_U,on = ['User_id'], how = 'left')
dftest = pd.merge(dftest,DistanceFilling_M,on = ['Merchant_id'], how = 'left')


In [69]:
def DistanceMissingFill(data):
    if np.isnan(data['Distance']):
        if not np.isnan(data['DistanceFilling_UM']):
            return int(data['DistanceFilling_UM'])
        elif not np.isnan(data['DistanceFilling_U']):
            return int(data['DistanceFilling_U'])
        elif not  np.isnan(data['DistanceFilling_M']):
            return int(data['DistanceFilling_M'])
    return data['Distance']

dfoff['Distance'] = dfoff.apply(DistanceMissingFill, axis = 1)
dftest['Distance'] = dftest.apply(DistanceMissingFill, axis = 1)

In [70]:
## Creat target label 
"""
According to the definition, 
1) buy with coupon within (include) 15 days ==> 1
2) buy with coupon but out of 15 days ==> 0
3) buy without coupon ==> -1 (we don't care)
"""
def label(row):
    if np.isnan(row['Date_received']):
        return 0
    if not np.isnan(row['Date']):
        td = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0

dfoff["label"] = dfoff.apply(label, axis=1)
dfoff["label"].value_counts()

0    1124438
1      36304
Name: label, dtype: int64

In [71]:
def getWeekday(row):
    if (np.isnan(row)) or (row==-1):
        return row
    else:
        return pd.to_datetime(row, format = "%Y%m%d").dayofweek+1 # add one to make it from 0~6 -> 1~7

dfoff['weekday'] = dfoff['Date_received'].apply(getWeekday)
dftest['weekday'] = dftest['Date_received'].apply(getWeekday)
# weekday_type (weekend = 1)
dfoff['weekday_type'] = dfoff['weekday'].apply(lambda x : 1 if x in [6,7] else 0 ) # apply to trainset
dftest['weekday_type'] = dftest['weekday'].apply(lambda x : 1 if x in [6,7] else 0 ) # apply to testset

In [72]:
# 確認消費時間是否為 4 個假期
def check_new_year(x):
    start = datetime.date(2016, 2, 3)
    end = datetime.date(2016, 2, 13)
    if (np.isnan(x)) or (x==-1):
        return 0
    else:
        if start <= pd.to_datetime(x, format = "%Y%m%d") <= end:
            return 1
        else:
            return 0
def check_child(x):
    start = datetime.date(2016, 4, 2)
    end = datetime.date(2016, 4, 5)
    if (np.isnan(x)) or (x==-1):
        return 0
    else:
        if start <= pd.to_datetime(x, format = "%Y%m%d") <= end:
            return 1
        else:
            return 0
def check_mother(x):
    start = datetime.date(2016, 5, 1)
    end = datetime.date(2016, 5, 8)
    if (np.isnan(x)) or (x==-1):
        return 0
    else:
        if start <= pd.to_datetime(x, format = "%Y%m%d") <= end:
            return 1
        else:
            return 0       

dfoff['new_year'] = dfoff['Date_received'].apply(check_new_year)
dfoff['child'] = dfoff['Date_received'].apply(check_child)
dfoff['mother'] = dfoff['Date_received'].apply(check_mother)



In [73]:
dftest['new_year'] = dftest['Date_received'].apply(check_new_year)
dftest['child'] = dftest['Date_received'].apply(check_child)
dftest['mother'] = dftest['Date_received'].apply(check_mother)

In [74]:
# Generate features - coupon discount and distance
def getDiscountType(row):
    if np.isnan(float(row.replace(':',''))):
        return 0
    else:
        return 1

def convertRate(row):
    """Convert discount to rate"""
    if np.isnan(float(row.replace(':',''))):
        return 1
    elif ':' in row:
        rows = row.split(':')
        return 1.0 - float(rows[1])/float(rows[0])
    else:
        return float(row)

def getDiscountJian(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0
def getDiscountMan(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0
    
def processData(df):
    
    # convert discunt_rate
    df['discount_rate'] = df['Discount_rate'].astype('str').apply(convertRate)
    df['discount_man'] = df['Discount_rate'].astype('str').apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].astype('str').apply(getDiscountJian)
    df['discount_type'] = df['Discount_rate'].astype('str').apply(getDiscountType)
    
    # convert distance
    df.loc[df.Distance.isna(), "Distance"] = 99
    
            
            
    
    return df

dfoff = processData(dfoff)
dftest = processData(dftest)

In [75]:
cnn_feature = ['discount_rate',
                    'discount_type',
                    'discount_jian',
                    'discount_man',
                    'Distance', 
                    'weekday_type',
                    'new_year',
                    'dragon',
                    'mother',
                    'child',]

In [76]:
training_X = dfoff[cnn_feature].values
training_Y = dfoff['label'].values

In [188]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical
from keras.optimizers import Adam
from sklearn.metrics import roc_auc_score
from keras import metrics

model = Sequential()
model.add(Dense(128, input_dim=10, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer = 'adam',
              metrics = [metrics.categorical_accuracy])

es = EarlyStopping(monitor='val_loss', patience=5)

model.fit(training_X, to_categorical(training_Y),
          epochs=100,
          batch_size=1024,
          shuffle=True,
          validation_split=0.5,
          callbacks=[es])

Train on 580371 samples, validate on 580371 samples
Epoch 1/100
580371/580371 [==============================] - 6s 11us/step - loss: 0.1369 - categorical_accuracy: 0.9680 - val_loss: 0.1072 - val_categorical_accuracy: 0.9687
Epoch 2/100
580371/580371 [==============================] - 5s 9us/step - loss: 0.1090 - categorical_accuracy: 0.9688 - val_loss: 0.1066 - val_categorical_accuracy: 0.9687
Epoch 3/100
580371/580371 [==============================] - 5s 8us/step - loss: 0.1072 - categorical_accuracy: 0.9688 - val_loss: 0.1067 - val_categorical_accuracy: 0.9687
Epoch 4/100
580371/580371 [==============================] - 4s 7us/step - loss: 0.1066 - categorical_accuracy: 0.9688 - val_loss: 0.1052 - val_categorical_accuracy: 0.9687
Epoch 5/100
580371/580371 [==============================] - 5s 8us/step - loss: 0.1061 - categorical_accuracy: 0.9688 - val_loss: 0.1050 - val_categorical_accuracy: 0.9687
Epoch 6/100
580371/580371 [==============================] - 5s 9us/step - loss: 0

KeyboardInterrupt: 

In [182]:

test_X = dftest[cnn_feature].values
results = model.predict(test_X)


In [183]:
targetset = dftest.copy()
testset = targetset[cnn_feature].copy()
test1 = testset.copy()

test1['pred_prob'] = results[:, 1]

In [184]:
output = pd.concat((targetset[["User_id", "Coupon_id", "Date_received"]], test1["pred_prob"]), axis=1)
print(output.shape)

output.loc[:, "User_id"] = output["User_id"].apply(lambda x:str(int(x)))
output.loc[:, "Coupon_id"] = output["Coupon_id"].apply(lambda x:str(int(x)))
output.loc[:, "Date_received"] = output["Date_received"].apply(lambda x:str(int(x)))
output["uid"] = output[["User_id", "Coupon_id", "Date_received"]].apply(lambda x: '_'.join(x.values), axis=1)
output.reset_index(drop=True, inplace=True)

(306313, 4)


In [185]:
out = output.groupby("uid", as_index=False).mean()
out = out[["uid", "pred_prob"]]
out.columns = ["uid", "label"]
out.to_csv("cnn_adam_2.csv", header=["uid", "label"], index=False) # submission format
out.head()

,uid,label
0,1000020_2705_20160519,0.233459
1,1000020_8192_20160513,0.233459
2,1000065_1455_20160527,0.110296
3,1000085_8067_20160513,0.140469
4,1000086_2418_20160613,0.140469


In [82]:

def tuneParamsRandom(classifier, params, train_x, train_y, cv = 5):
    rs = RandomizedSearchCV(classifier, params, n_iter = 20, scoring = 'roc_auc', n_jobs = -1, verbose = 0, cv=cv)
    rs.fit(train_x, train_y)
    
    return rs.best_params_, abs(rs.best_score_)

In [83]:
etParams = {'n_estimators':np.arange(100,1100,50), 'max_depth':np.arange(3,11,2),
            'min_samples_leaf':np.arange(1,6,2) }

rfParams = {'n_estimators':np.arange(100,1100,50), 'max_depth':np.arange(3,11,2),
            'min_samples_split':np.arange(2,20,2), 'min_samples_leaf':np.arange(1,6,2) }

abParams = {'n_estimators':np.arange(100,1100,50),'learning_rate':np.arange(0.01,0.2,0.05)}


lgbmcParams = {'n_estimators' : np.arange(400,1200,100), 'learning_rate' : np.arange(0.01,0.1,0.02),
               'num_leaves' : np.arange(2,48,4), 'max_depth' : np.arange(3,10,2),
               'subsample' : np.arange(0.3,0.8,0.1) }

gbcParams = {'n_estimators' : np.arange(400,1200,100) , 'learning_rate' : np.arange(0.01,0.1,0.02) 
             , 'min_samples_split' : np.arange(2,30,5), 'min_samples_leaf' : np.arange(2,32,4),
              'max_depth' : np.arange(3,8,2), 'subsample' : np.arange(0.3,0.8,0.1)}

xgbParams = {'max_depth':np.arange(3,8,1),'learning_rate':np.arange(0.01,0.1,0.02)
             ,'n_estimator': np.arange(1000,3000,100),'gamma':np.arange(0.01,0.1,0.02)}

lrParams = {'C':np.arange(0.01,1,0.05), 'max_iter' : np.arange(100,500,100)}

In [132]:
train_label = dfoff.label
train_x = dfoff[cnn_feature]

train_label.index = train_x.index
tuneSet = pd.concat([train_x,train_label], axis = 1)
trainSet = tuneSet.sample(frac=0.5)
trainSet.shape

(580371, 11)

In [134]:
x_train, x_test, y_train, y_test = train_test_split(trainSet[trainSet.columns[trainSet.columns != 'label']],trainSet['label'], test_size = 0.3, random_state = 1234)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(406259, 10)
(406259,)
(174112, 10)
(174112,)


In [94]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor
from sklearn.linear_model import LogisticRegression
from mlxtend.regressor import StackingRegressor
import sklearn.metrics
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV


In [135]:
gbc = GradientBoostingClassifier(max_features = 'sqrt')
gbc_best_Params,gbc_best_score = tuneParamsRandom(gbc,gbcParams,x_train,y_train)
print("GradientBoostingClassifier:",gbc_best_Params,gbc_best_score)


gbc = GradientBoostingClassifier(**gbc_best_Params,max_features = 'sqrt')
gbc.fit(train_x, train_label)
gbc_pred = gbc.predict_proba(test_X)[:,1]

importances = pd.DataFrame(gbc.feature_importances_, columns = ['importances'], index = train_x.columns)
importances.importances.sort_values(ascending = False)

GradientBoostingClassifier: {'subsample': 0.4, 'n_estimators': 500, 'min_samples_split': 17, 'min_samples_leaf': 18, 'max_depth': 5, 'learning_rate': 0.049999999999999996} 0.8886836355236886


Distance         0.250913
discount_man     0.245383
discount_rate    0.224453
discount_type    0.122303
discount_jian    0.119005
weekday_type     0.017789
new_year         0.015599
child            0.004555
mother           0.000000
dragon           0.000000
Name: importances, dtype: float64

In [136]:
lgbmc_best_Params, lgbmc_best_score = tuneParamsRandom(LGBMClassifier(),lgbmcParams,x_train,y_train)
print("LGBMClassifier:",lgbmc_best_Params,lgbmc_best_score)


lgbmc = LGBMClassifier(**lgbmc_best_Params)
lgbmc.fit(train_x,train_label)
lgbmc_pred = lgbmc.predict_proba(test_X)[:,1]

LGBMClassifier: {'subsample': 0.5, 'num_leaves': 22, 'n_estimators': 800, 'max_depth': 3, 'learning_rate': 0.08999999999999998} 0.8883883312041179


In [137]:
xgb_best_Params,xgb_best_score = tuneParamsRandom(XGBClassifier(),xgbParams, x_train, y_train)
print("XGBClassifier:",xgb_best_Params,xgb_best_score)

xgb = XGBClassifier(**xgb_best_Params)
xgb.fit(train_x,train_label)
xgb_pred = xgb.predict_proba(test_X)[:,1]

XGBClassifier: {'n_estimator': 2700, 'max_depth': 6, 'learning_rate': 0.08999999999999998, 'gamma': 0.049999999999999996} 0.8885360141291645


ValueError: feature_names mismatch: ['discount_rate', 'discount_type', 'discount_jian', 'discount_man', 'Distance', 'weekday_type', 'new_year', 'dragon', 'mother', 'child'] ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9']
expected child, dragon, discount_type, new_year, discount_rate, weekday_type, Distance, mother, discount_man, discount_jian in input data
training data did not have the following fields: f7, f2, f9, f5, f0, f1, f6, f8, f4, f3

In [141]:
test_X = dftest[dfoff[cnn_feature].columns]


In [142]:
xgb = XGBClassifier(**xgb_best_Params)
xgb.fit(train_x,train_label)
xgb_pred = xgb.predict_proba(test_X)[:,1]

In [155]:
blending_pred = gbc_pred * 0.34 + lgbmc_pred * 0.33 + xgb_pred * 0.33

In [144]:
from mlxtend.classifier import StackingClassifier
gbc = GradientBoostingClassifier(**gbc_best_Params)
lgbmc = LGBMClassifier(**lgbmc_best_Params)
xgb = XGBClassifier(**xgb_best_Params)

#gbc = GradientBoostingClassifier()
#lgbmc = LGBMClassifier()
#xgb = XGBClassifier()

meta_estimator = LGBMClassifier()
stacking = StackingClassifier(classifiers = [gbc,lgbmc, xgb], meta_classifier = meta_estimator)
stacking.fit(x_train,y_train)
stacking_pred = stacking.predict_proba(test_X)[:,1]

In [156]:
test1['pred_prob'] = blending_pred

In [157]:
output = pd.concat((targetset[["User_id", "Coupon_id", "Date_received"]], test1["pred_prob"]), axis=1)
print(output.shape)

output.loc[:, "User_id"] = output["User_id"].apply(lambda x:str(int(x)))
output.loc[:, "Coupon_id"] = output["Coupon_id"].apply(lambda x:str(int(x)))
output.loc[:, "Date_received"] = output["Date_received"].apply(lambda x:str(int(x)))
output["uid"] = output[["User_id", "Coupon_id", "Date_received"]].apply(lambda x: '_'.join(x.values), axis=1)
output.reset_index(drop=True, inplace=True)

(306313, 4)


In [158]:
out = output.groupby("uid", as_index=False).mean()
out = out[["uid", "pred_prob"]]
out.columns = ["uid", "label"]
out.to_csv("blending_pred.csv", header=["uid", "label"], index=False) # submission format
out.head()

,uid,label
0,1000020_2705_20160519,0.246100
1,1000020_8192_20160513,0.246100
2,1000065_1455_20160527,0.128948
3,1000085_8067_20160513,0.142277
4,1000086_2418_20160613,0.142277
